In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# ============================================
# 0. CONFIG
# ============================================

TRAIN_PATH = "train.csv"
TEST_PATH = "test_for_participants.csv"
SAMPLE_SUB_PATH = "sample_submission.csv"
OUTPUT_SUB_PATH = "xgb_ensemble_top3_residual_submission.csv"

TARGET_COL = "target"
DATE_COL = "delivery_start"

# Rolling CV config: 7 months train, 1 month val
TRAIN_MONTHS = 7
VAL_MONTHS = 1

# Hyperparameter search config
N_ITER = 15
RANDOM_STATE = 42

# Base TRAIN-ONLY trimming
BASE_TRIM_Q_LOW = 0.01
BASE_TRIM_Q_HIGH = 0.99

# Extra TRAINING-ONLY trimming options inside base-trimmed data
EXTRA_TAIL_TRIM_OPTIONS = [
    (0.00, 1.00),# no extra trimming
    #(0.01,0.99),
    #(0.02,0.98)
    
]

# Logging config for debugging learning curves
LOG_CURVES = False         
MAX_LOG_FOLDS = 3       

# ============================================
# 1. FEATURE ENGINEERING
# ============================================

def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df[DATE_COL] = pd.to_datetime(df[DATE_COL])

    # --- TIME FEATURES ---
    df["hour"] = df[DATE_COL].dt.hour
    df["day_of_week"] = df[DATE_COL].dt.dayofweek
    df["month"] = df[DATE_COL].dt.month
    df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

    df["year"] = df[DATE_COL].dt.year

    df["is_2023"] = (df["year"] == 2023).astype(int)
    df["is_2024"] = (df["year"] == 2024).astype(int)
    df["is_2025"] = (df["year"] == 2025).astype(int)

    df["is_first_half"] = (df["month"] <= 6).astype(int)
    df["is_second_half"] = (df["month"] >= 7).astype(int)

    # Cyclical encodings
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)

    df["dow_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
    df["dow_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)

    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

    df["day_of_year"] = df[DATE_COL].dt.dayofyear
    df["doy_sin"] = np.sin(2 * np.pi * df["day_of_year"] / 365)
    df["doy_cos"] = np.cos(2 * np.pi * df["day_of_year"] / 365)

    df["is_peak_hour"] = df["hour"].isin([7, 8, 9, 17, 18, 19]).astype(int)
    df["is_night"] = df["hour"].isin([0, 1, 2, 3, 4]).astype(int)

    df["is_winter"] = df["month"].isin([12, 1, 2]).astype(int)
    df["is_summer"] = df["month"].isin([6, 7, 8]).astype(int)

    # --- WIND DIRECTION ---
    df["wind_dir_rad"] = np.deg2rad(df["wind_direction_80m"])
    df["wind_dir_sin"] = np.sin(df["wind_dir_rad"])
    df["wind_dir_cos"] = np.cos(df["wind_dir_rad"])

    # --- NET LOAD & RENEWABLES ---
    df["net_load"] = df["load_forecast"] - (df["solar_forecast"] + df["wind_forecast"])

    df["renewable_share"] = (df["solar_forecast"] + df["wind_forecast"]) / (df["load_forecast"] + 1.0)
    df["net_load_squared"] = df["net_load"] ** 2

    # IMPORTANT: sort by market + time so ramps are chronological within each market
    df = df.sort_values(["market", DATE_COL]).reset_index(drop=True)

    # Ramps grouped by market
    df["net_load_ramp"] = df.groupby("market")["net_load"].diff()
    df["load_ramp"] = df.groupby("market")["load_forecast"].diff()
    df["load_ramp_3h"] = df.groupby("market")["load_forecast"].diff(3)

    df["net_load_ramp"] = df["net_load_ramp"].fillna(0)
    df["load_ramp"] = df["load_ramp"].fillna(0)
    df["load_ramp_3h"] = df["load_ramp_3h"].fillna(0)

    df["net_load_peak"] = df["net_load"] * df["is_peak_hour"]
    df["is_high_renewable"] = (df["renewable_share"] > df["renewable_share"].median()).astype(int)

    df["scarcity_score"] = (
        df["net_load"] - df["net_load"].mean()
    ) / (df["net_load"].std() + 1e-6)

    # --- WIND FEATURES ---
    def wind_power_smooth(v, c=1500):
        v3 = v ** 3
        return v3 / (v3 + c)

    df["wind_power_shape"] = wind_power_smooth(df["wind_speed_80m"])

    wind_scale = df["wind_forecast"].mean()
    df["wind_actual_proxy_MW"] = df["wind_power_shape"] * wind_scale

    df["wind_surprise_MW"] = df["wind_forecast"] - df["wind_actual_proxy_MW"]
    df["wind_surprise_relative"] = df["wind_surprise_MW"] / (df["wind_forecast"] + 1.0)
    df["wind_surprise_sign"] = np.sign(df["wind_surprise_MW"])
    df["wind_proxy_sq"] = df["wind_actual_proxy_MW"] ** 2
    df["wind_peak"] = df["wind_actual_proxy_MW"] * df["is_peak_hour"]
    df["wind_winter"] = df["wind_actual_proxy_MW"] * df["is_winter"]

    # Wind ramp grouped by market
    df["wind_ramp"] = df.groupby("market")["wind_actual_proxy_MW"].diff()
    df["wind_ramp"] = df["wind_ramp"].fillna(0)

    # --- SOLAR FEATURES ---
    df["solar_power_shape"] = (
        0.75 * df["direct_normal_irradiance"] +
        0.25 * df["diffuse_horizontal_irradiance"]
    )
    df["solar_power_shape"] /= df["solar_power_shape"].max() + 1e-6

    if "air_temperature_2m" in df.columns:
        df["solar_temp_factor"] = 1 - 0.004 * (df["air_temperature_2m"] - 25)
        df["solar_temp_factor"] = df["solar_temp_factor"].clip(0.8, 1.05)
        df["solar_power_shape"] *= df["solar_temp_factor"]

    solar_scale = df["solar_forecast"].mean()
    df["solar_actual_proxy_MW"] = df["solar_power_shape"] * solar_scale

    df["solar_surprise_MW"] = df["solar_forecast"] - df["solar_actual_proxy_MW"]
    df["solar_surprise_relative"] = df["solar_surprise_MW"] / (df["solar_forecast"] + 1.0)
    df["solar_surprise_sign"] = np.sign(df["solar_surprise_MW"])

    # Solar ramp grouped by market
    df["solar_ramp"] = df.groupby("market")["solar_actual_proxy_MW"].diff()
    df["solar_ramp"] = df["solar_ramp"].fillna(0)

    # System ramp
    df["system_ramp"] = df["load_ramp"] - df["wind_ramp"] - df["solar_ramp"]

    # Encode market
    if "market" in df.columns:
        df = pd.get_dummies(df, columns=["market"], drop_first=True)

    # Drop raw time / direction columns (keep DATE_COL)
    drop_cols = ["hour", "day_of_week", "month", "day_of_year", "wind_direction_80m", "wind_dir_rad"]
    drop_cols = [c for c in drop_cols if c in df.columns]
    df = df.drop(columns=drop_cols)

    return df

# ============================================
# 2. TIME-BASED ROLLING SPLITS (NO TRIM HERE)
# ============================================

def build_time_splits(dates: pd.Series, train_months=6, val_months=1):
    dates = pd.to_datetime(dates)
    months = dates.dt.to_period("M")
    unique_months = np.array(sorted(months.unique()))

    splits = []
    total = train_months + val_months

    for start in range(len(unique_months) - total + 1):
        train_m = unique_months[start: start + train_months]
        val_m = unique_months[start + train_months: start + total]

        train_idx = np.where(months.isin(train_m))[0]
        val_idx = np.where(months.isin(val_m))[0]

        if len(train_idx) == 0 or len(val_idx) == 0:
            continue

        splits.append((train_idx, val_idx))

    return splits

# ============================================
# 3. HYPERPARAMETER SAMPLING
# ============================================

def sample_xgb_params(rng: np.random.RandomState):
    params = {
        "n_estimators": rng.randint(1200, 7000),
        "max_depth": rng.randint(4, 8),
        "learning_rate": 10 ** rng.uniform(-1.5, -0.7),         # ~0.1–0.316
        "subsample": rng.uniform(0.6, 0.9),
        "colsample_bytree": rng.uniform(0.6, 0.9),
        "min_child_weight": rng.uniform(5.0, 30.0),
        "gamma": 10 ** rng.uniform(-1.5, 0),                  # ~0.03–1
        "reg_alpha": 10 ** rng.uniform(-2, 1),                # 0.01–10
        "reg_lambda": 10 ** rng.uniform(-1, 2),               # 0.1–100
        "tree_method": "hist",
        "objective": "reg:squarederror",
        "random_state": RANDOM_STATE,
        "n_jobs": -1,
        "eval_metric": "rmse",
    }
    return params

# ============================================
# 4. EVALUATE ONE CONFIG (TRAIN-ONLY TRIM, RESIDUAL TARGET)
# ============================================

def evaluate_config(
    X_all_np,
    y_all_np,
    splits,
    params,
    extra_lower_q,
    extra_upper_q,
    early_stopping_rounds=150,
    baseline_col_idx=None,
):
    rmses = []
    fold_id = 0

    for (train_idx, val_idx) in splits:
        fold_id += 1

        X_train_full = X_all_np[train_idx]
        y_train_full = y_all_np[train_idx]

        X_val = X_all_np[val_idx]
        y_val = y_all_np[val_idx]  # original prices

        # Baseline from mh_price_mean column
        if baseline_col_idx is None:
            raise ValueError("baseline_col_idx must be provided for residual modeling")

        baseline_train = X_train_full[:, baseline_col_idx]
        baseline_val = X_val[:, baseline_col_idx]

        # Base train-only trimming (1–99%) on ORIGINAL y
        q_low_base = np.quantile(y_train_full, BASE_TRIM_Q_LOW)
        q_high_base = np.quantile(y_train_full, BASE_TRIM_Q_HIGH)
        mask_base = (y_train_full >= q_low_base) & (y_train_full <= q_high_base)

        y_train_base = y_train_full[mask_base]
        X_train_base = X_train_full[mask_base]
        baseline_train_base = baseline_train[mask_base]

        # Residual target after base trim
        y_train_base_res = y_train_base - baseline_train_base

        # Extra TRAINING-ONLY trimming inside base (still based on ORIGINAL y)
        if extra_lower_q > 0.0 or extra_upper_q < 1.0:
            q_low_extra = np.quantile(y_train_base, extra_lower_q)
            q_high_extra = np.quantile(y_train_base, extra_upper_q)
            mask_extra = (y_train_base >= q_low_extra) & (y_train_base <= q_high_extra)

            X_train = X_train_base[mask_extra]
            y_train_res = y_train_base_res[mask_extra]
        else:
            X_train = X_train_base
            y_train_res = y_train_base_res

        if len(y_train_res) < 200:
            continue

        # Residuals on val (for early stopping metric)
        y_val_res = y_val - baseline_val

        model = XGBRegressor(
            **params,
            early_stopping_rounds=early_stopping_rounds,
        )

        model.fit(
            X_train,
            y_train_res,
            eval_set=[(X_train, y_train_res), (X_val, y_val_res)],
            verbose=False,
        )

        # Logging (now residual RMSE curves)
        if LOG_CURVES and fold_id <= MAX_LOG_FOLDS:
            evals = model.evals_result()
            train_curve = evals["validation_0"]["rmse"]
            val_curve = evals["validation_1"]["rmse"]

            n_rounds = len(train_curve)
            best_iter = int(np.argmin(val_curve))
            best_train = train_curve[best_iter]
            best_val = val_curve[best_iter]

            last_train = train_curve[-1]
            last_val = val_curve[-1]

            print(f"\n=== Fold {fold_id} learning curves (residual target) ===")
            print(f"  Rounds: {n_rounds}")
            print(f"  Best iter: {best_iter} | train_rmse_res={best_train:.3f}, val_rmse_res={best_val:.3f}")
            print(f"  Last iter: {n_rounds-1} | train_rmse_res={last_train:.3f}, val_rmse_res={last_val:.3f}")

            for check in [0, 10, 50, 100, 200, 500, best_iter]:
                if 0 <= check < n_rounds:
                    print(
                        f"    iter {check:4d}: "
                        f"train_res={train_curve[check]:.3f}, val_res={val_curve[check]:.3f}"
                    )

        # ---- Evaluate in ORIGINAL price space ----
        y_pred_res = model.predict(X_val)
        y_pred_full = y_pred_res + baseline_val

        rmse = np.sqrt(mean_squared_error(y_val, y_pred_full))
        rmses.append(rmse)

    if len(rmses) == 0:
        return np.inf

    return float(np.mean(rmses))

# ============================================
# 5. MAIN
# ============================================

def main():
    rng = np.random.RandomState(RANDOM_STATE)

    # --- Load data ---
    train = pd.read_csv(TRAIN_PATH)
    test = pd.read_csv(TEST_PATH)
    sample_sub = pd.read_csv(SAMPLE_SUB_PATH)

    # --- Cut 2023 H2 (Jul–Dec 2023) from TRAIN ---
    train[DATE_COL] = pd.to_datetime(train[DATE_COL])
    mask_keep = ~(
        (train[DATE_COL].dt.year == 2023) & (train[DATE_COL].dt.month >= 7)
    )
    print(f"Rows before 2023 H2 cut: {len(train)}")
    train = train.loc[mask_keep].reset_index(drop=True)
    print(f"Rows after  2023 H2 cut: {len(train)}")

    # Make sure DATE_COL in test is datetime too
    test[DATE_COL] = pd.to_datetime(test[DATE_COL])

    # ============================================
    # Market + hour price profile (from TRAIN ONLY)
    # ============================================

    train["profile_hour"] = train[DATE_COL].dt.hour
    test["profile_hour"] = test[DATE_COL].dt.hour

    mh_profile = (
        train.groupby(["market", "profile_hour"])[TARGET_COL]
        .agg(["mean", "median"])
        .reset_index()
        .rename(columns={"mean": "mh_price_mean", "median": "mh_price_median"})
    )

    train = train.merge(mh_profile, on=["market", "profile_hour"], how="left")
    test = test.merge(mh_profile, on=["market", "profile_hour"], how="left")

    train = train.drop(columns=["profile_hour"])
    test = test.drop(columns=["profile_hour"])

    # --- Feature engineering ---
    train_fe = engineer_features(train)
    test_fe = engineer_features(test)

    # --- Handle missing numeric values using train medians ---
    numeric_cols = train_fe.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [c for c in numeric_cols if c not in [TARGET_COL, "id"]]

    median_dict = train_fe[numeric_cols].median()
    train_fe[numeric_cols] = train_fe[numeric_cols].fillna(median_dict)

    for col in median_dict.index:
        if col in test_fe.columns:
            test_fe[col] = test_fe[col].fillna(median_dict[col])

    # --- Build feature list (no global trimming here) --
    non_feature_cols = [TARGET_COL, DATE_COL]
    if "delivery_end" in train_fe.columns:
        non_feature_cols.append("delivery_end")
    if "id" in train_fe.columns:
        non_feature_cols.append("id")

    candidate_feature_cols = [
        c for c in train_fe.columns if c not in non_feature_cols
    ]
    feature_cols = [
        c for c in candidate_feature_cols
        if pd.api.types.is_numeric_dtype(train_fe[c])
    ]

    # Index of baseline feature for residual modeling
    assert "mh_price_mean" in feature_cols, "mh_price_mean must be in feature_cols for residual modeling"
    baseline_col_idx = feature_cols.index("mh_price_mean")

    print(f"Number of features: {len(feature_cols)}")

    X_df = train_fe[feature_cols].reset_index(drop=True)
    y_series = train_fe[TARGET_COL].reset_index(drop=True)

    X_all_np = X_df.values
    y_all_np = y_series.values

    # --- Rolling splits on FULL (UNTRIMMED) data ---
    splits = build_time_splits(train_fe[DATE_COL], TRAIN_MONTHS, VAL_MONTHS)
    print(f"Number of rolling splits: {len(splits)}")

    # ============================================
    # 6. RANDOM SEARCH (TRAIN-ONLY TRIM, UNTRIMMED VAL, RESIDUAL TARGET)
    # ============================================

    best_score = np.inf
    best_params = None
    best_extra_trim = None

    all_configs = []   # store (score, params, extra_trim)

    for i in range(1, N_ITER + 1):
        params = sample_xgb_params(rng)
        extra_lower_q, extra_upper_q = EXTRA_TAIL_TRIM_OPTIONS[
            rng.randint(len(EXTRA_TAIL_TRIM_OPTIONS))
        ]

        mean_rmse = evaluate_config(
            X_all_np,
            y_all_np,
            splits,
            params,
            extra_lower_q,
            extra_upper_q,
            early_stopping_rounds=50,
            baseline_col_idx=baseline_col_idx,
        )

        all_configs.append({
            "score": mean_rmse,
            "params": params,
            "extra_trim": (extra_lower_q, extra_upper_q),
        })

        print(
            f"[{i}/{N_ITER}] "
            f"CV RMSE={mean_rmse:.3f} | extra_trim=({extra_lower_q:.3f}, {extra_upper_q:.3f}) "
            f"| n_estimators={params['n_estimators']}, "
            f"max_depth={params['max_depth']}, "
            f"lr={params['learning_rate']:.4f}"
        )

        if mean_rmse < best_score:
            best_score = mean_rmse
            best_params = params
            best_extra_trim = (extra_lower_q, extra_upper_q)
            print(
                f"  -> New BEST: CV RMSE={best_score:.3f}, "
                f"extra_trim=({extra_lower_q:.3f}, {extra_upper_q:.3f})"
            )

    print("\n===== BEST CONFIGURATION FOUND (SINGLE) =====")
    print(f"Best mean CV RMSE: {best_score:.3f}")
    print(f"Best extra training trim quantiles: {best_extra_trim}")
    print("Best params:")
    for k, v in best_params.items():
        print(f"  {k}: {v}")

    # ENSEMBLE: pick top-3 configs by CV RMSE
    all_configs_sorted = sorted(all_configs, key=lambda c: c["score"])
    top_k = min(3, len(all_configs_sorted))
    top_configs = all_configs_sorted[:top_k]

    print(f"\n===== TOP {top_k} CONFIGURATIONS FOR ENSEMBLE =====")
    for rank, cfg in enumerate(top_configs, start=1):
        print(f"\n-- Model {rank} --")
        print(f"CV RMSE: {cfg['score']:.3f}")
        print(f"extra_trim: {cfg['extra_trim']}")
        for k, v in cfg["params"].items():
            print(f"  {k}: {v}")

    # ============================================
    # Prepare TEST FEATURES once (for all ensemble models)
    # ============================================

    test_fe = sample_sub[["id"]].merge(test_fe, on="id", how="left")

    X_test = test_fe.reindex(columns=feature_cols)
    X_test = X_test.fillna(median_dict)
    X_test = X_test.fillna(0.0)

    baseline_test = X_test.iloc[:, baseline_col_idx].values  # mh_price_mean for test

    # ============================================
    # 7. TRAIN FINAL TOP-K MODELS & PREDICT ON TEST (RESIDUAL)
    # ============================================

    months_all = train_fe[DATE_COL].dt.to_period("M")
    final_val_month = months_all.max()

    val_mask_final = months_all == final_val_month
    train_mask_final = ~val_mask_final

    X_train_full_global = X_df.loc[train_mask_final].reset_index(drop=True)
    y_train_full_global = y_series.loc[train_mask_final].reset_index(drop=True)

    X_val_final_global = X_df.loc[val_mask_final].reset_index(drop=True)
    y_val_final_global = y_series.loc[val_mask_final].reset_index(drop=True)

    print(
        f"\nFinal split for training (shared): "
        f"{train_mask_final.sum()} rows train, "
        f"{val_mask_final.sum()} rows validation "
        f"(val month = {final_val_month})"
    )

    test_pred_list = []

    for rank, cfg in enumerate(top_configs, start=1):
        params_m = cfg["params"]
        extra_lower_q_m, extra_upper_q_m = cfg["extra_trim"]

        print(f"\n=== Training final model {rank} / {top_k} ===")
        print(f"Using extra_trim = ({extra_lower_q_m:.3f}, {extra_upper_q_m:.3f})")

        X_train_full = X_train_full_global.copy()
        y_train_full = y_train_full_global.copy()
        X_val_final = X_val_final_global.copy()
        y_val_final = y_val_final_global.copy()

        baseline_train_full = X_train_full.iloc[:, baseline_col_idx].values
        baseline_val = X_val_final.iloc[:, baseline_col_idx].values

        # Base train-only trimming on original y
        q_low_base = np.quantile(y_train_full.values, BASE_TRIM_Q_LOW)
        q_high_base = np.quantile(y_train_full.values, BASE_TRIM_Q_HIGH)
        mask_base_final = (y_train_full >= q_low_base) & (y_train_full <= q_high_base)

        y_train_base_final = y_train_full[mask_base_final]
        X_train_base_final = X_train_full[mask_base_final]
        baseline_train_base = baseline_train_full[mask_base_final]

        # Residual targets after base trim
        y_train_base_res = y_train_base_final.values - baseline_train_base

        if extra_lower_q_m > 0.0 or extra_upper_q_m < 1.0:
            q_low_extra = np.quantile(y_train_base_final.values, extra_lower_q_m)
            q_high_extra = np.quantile(y_train_base_final.values, extra_upper_q_m)
            mask_extra_final = (y_train_base_final >= q_low_extra) & (y_train_base_final <= q_high_extra)

            X_train_final = X_train_base_final[mask_extra_final]
            y_train_final_res = y_train_base_res[mask_extra_final]

            print(
                f"  Final training: extra trimming kept "
                f"{mask_extra_final.sum()} / {len(mask_extra_final)} training rows "
                f"with target in [{q_low_extra:.2f}, {q_high_extra:.2f}]"
            )
        else:
            X_train_final = X_train_base_final
            y_train_final_res = y_train_base_res
            print(
                f"  Final training: base 1–99% trimming kept "
                f"{len(y_train_final_res)} / {len(y_train_full)} training rows "
                f"with target in [{q_low_base:.2f}, {q_high_base:.2f}]"
            )

        model_m = XGBRegressor(**params_m, early_stopping_rounds=50)

        y_val_final_res = y_val_final.values - baseline_val

        model_m.fit(
            X_train_final.values,
            y_train_final_res,
            eval_set=[(X_train_final.values, y_train_final_res),
                      (X_val_final.values, y_val_final_res)],
            verbose=False,
        )

        if hasattr(model_m, "best_iteration"):
            print(f"  Final model {rank} best_iteration (residual): {model_m.best_iteration}")
        else:
            print(f"  Final model {rank} trained (no best_iteration attribute).")

        # Predict residuals on test for this model, then add baseline
        test_pred_res_m = model_m.predict(X_test.values)
        test_pred_full_m = test_pred_res_m + baseline_test
        test_pred_list.append(test_pred_full_m)

    # ============================================
    # 8. ENSEMBLE: AVERAGE TOP-K TEST PREDICTIONS
    # ============================================

    test_pred_array = np.column_stack(test_pred_list)  # shape (n_test, top_k)
    ensemble_pred = test_pred_array.mean(axis=1)

    predictions = sample_sub.copy()
    predictions["target"] = ensemble_pred.astype(float)

    # ============================================
    # 9. VALIDATE SUBMISSION FORMAT & SAVE
    # ============================================

    assert list(predictions.columns) == ["id", "target"], "Wrong columns!"
    assert len(predictions) == 13098, f"Wrong row count: {len(predictions)}"
    assert predictions["id"].min() == 133627, "IDs must start at 133627"
    assert predictions["id"].max() == 146778, "IDs must end at 146778"
    assert predictions["target"].notna().all(), "No NaN values allowed!"
    assert np.isfinite(predictions["target"]).all(), "No infinite values allowed!"

    predictions.to_csv(OUTPUT_SUB_PATH, index=False)
    print(f"\n✅ Ensemble submission saved as {OUTPUT_SUB_PATH} with shape {predictions.shape}")
    print(f"   (Averaged over top {top_k} residual models by CV RMSE)")

if __name__ == "__main__":
    main()